In [1]:
import sys
sys.path.append('../')
from data.dataset import *
from evaluate import *
import tensorflow as tf
import tensorflow.keras.layers as tfkl
from tensorflow.keras.layers.experimental import preprocessing
import numpy as np
import os
import time

2021-08-16 19:44:53.639933: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


Cargo el dataset de texto y le aplico las transformaciones para poder usarlo para entrenar

In [2]:
path_to_file = '../data/txts/chamame_dataset.txt'

encoded_data, max_id, tokenizer = dataset_pre(path_to_file)
train_set, val_set = dataset_split(encoded_data, train_s = 90)
data_train = window_sequence(train_set, 100, 32, max_id, 1)
data_val = window_sequence(val_set, 100, 32, max_id, 512)

2021-08-16 19:44:54.381836: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-16 19:44:54.423851: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-16 19:44:54.424080: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1060 6GB computeCapability: 6.1
coreClock: 1.7715GHz coreCount: 10 deviceMemorySize: 5.93GiB deviceMemoryBandwidth: 178.99GiB/s
2021-08-16 19:44:54.424096: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-08-16 19:44:54.426870: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-08-16 19:44:54.426895: I tensorflow/stream_executo

Defino el modelo stateless. El recurrent drop out lo dejo en cero para evitar el warning de cuda

In [3]:
tf.keras.backend.clear_session()

entrada = tfkl.Input((None, max_id), name='ENTRADA')

capa_rec = tfkl.GRU(128, return_sequences=True, input_shape=[None, max_id],dropout=0.2, recurrent_dropout=0)(entrada)
capa_rec = tfkl.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0)(capa_rec)

salida = tfkl.TimeDistributed(tfkl.Dense(max_id, activation="softmax"))(capa_rec)

model = tf.keras.Model(inputs=[entrada], outputs=[salida])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")


Entrenamiento

In [4]:
history = model.fit(data_train, epochs=50)

2021-08-16 19:44:55.285019: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-16 19:44:55.301968: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3199980000 Hz


Epoch 1/50


2021-08-16 19:44:59.323332: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-08-16 19:44:59.587826: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8202
2021-08-16 19:44:59.687399: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11


      7/Unknown - 5s 10ms/step - loss: 3.7330

2021-08-16 19:44:59.876482: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


1247/1247 [==============================] - 19s 11ms/step - loss: 2.0977
Epoch 2/50
1247/1247 [==============================] - 15s 10ms/step - loss: 1.6827
Epoch 3/50
1247/1247 [==============================] - 16s 11ms/step - loss: 1.5191
Epoch 4/50
1247/1247 [==============================] - 18s 12ms/step - loss: 1.4182
Epoch 5/50
1247/1247 [==============================] - 17s 12ms/step - loss: 1.3475
Epoch 6/50
1247/1247 [==============================] - 17s 11ms/step - loss: 1.2948
Epoch 7/50
1247/1247 [==============================] - 17s 11ms/step - loss: 1.2525
Epoch 8/50
1247/1247 [==============================] - 18s 12ms/step - loss: 1.2186
Epoch 9/50
1247/1247 [==============================] - 17s 12ms/step - loss: 1.1913
Epoch 10/50
1247/1247 [==============================] - 16s 11ms/step - loss: 1.1688
Epoch 11/50
1247/1247 [==============================] - 20s 14ms/step - loss: 1.1475
Epoch 12/50
1247/1247 [==============================] - 19s 14ms/step - l

In [8]:
model.save_weights('pesos.hdf5')

Hago predicciones a partir del modelo entrenado

In [9]:
model.load_weights('pesos.hdf5')

In [18]:
print(complete_text(model, "Rio", tokenizer, n_chars=200, temperature=0.2))

Rio 
de la pier as solo ya tu sientra aquel palo de los pescadores. 



llevan a flor de agua su cantar  
no ambicionado soy  
hego e' macachín, miel de camachuí  
frito 'el ubajay, pisingallo, tas  
baq
